In [1]:
from tensorflow.keras.models import Model
import tensorflow as tf
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential,Model,initializers,layers
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.cifar10.load_data()

# Normalization
x_train=x_train/255.

x_test=x_test/255.

#One-hot encoding
y_train = tf.keras.utils.to_categorical(np.array(y_train))
y_test = tf.keras.utils.to_categorical(np.array(y_test))

datagen = ImageDataGenerator(rotation_range=8,
                             zoom_range=[0.95, 1.05],
                             height_shift_range=0.10,
                             shear_range=0.15)

In [4]:
x_train.shape

(50000, 32, 32, 3)

In [3]:
cfg = {
    'A': [64,     'M', 128,      'M', 256, 256,           'M', 512, 512,           'M', 512, 512,           'M'],
    'B': [64, 64, 'M', 128, 128, 'M', 256, 256,           'M', 512, 512,           'M', 512, 512,           'M'],
    'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256,      'M', 512, 512, 512,      'M', 512, 512, 512,      'M'],
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M']
}

In [4]:
class mymodel(Model):
    def __init__(self,features,num_classes,input_shape=(32,32,3)):
        super(mymodel,self).__init__()
        
        self.features=Sequential([
            layers.Input(input_shape),
            features
        ])
        
        
        self.classifier=Sequential([
            layers.Flatten(),
            layers.Dense(4096,activation='relu',kernel_initializer='he_normal'),
            layers.Dropout(0.5),
            layers.Dense(4096,activation='relu',kernel_initializer='he_normal'),
            layers.Dropout(0.5),
            layers.Dense(num_classes,activation='softmax')
        ])

    def call(self,inputs,training=True):
        x=self.features(inputs,training=training)
        x=self.classifier(x,training=training)
        return x

In [5]:
def make_layers(cfg):
    nets=[]
    for l in cfg:
        if l=='M':
            nets+=[layers.MaxPooling2D(pool_size=(2,2),padding='same')]
            continue
        
        nets+=[layers.Conv2D(filters=l,kernel_size=(3,3),padding='same')]
        nets+=[layers.BatchNormalization()]
        nets+=[layers.ReLU()]

    return Sequential(nets)

In [13]:
model=mymodel(make_layers(cfg['A']),10)

In [16]:
model.summary()

Model: "mymodel_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_13 (Sequential)  (None, 1, 1, 512)         9231488   
                                                                 
 sequential_14 (Sequential)  (None, 10)                18923530  
                                                                 
Total params: 28,155,018
Trainable params: 28,149,514
Non-trainable params: 5,504
_________________________________________________________________


In [14]:
lr_rate=tf.keras.optimizers.schedules.ExponentialDecay(1e-3, 10000, 0.95 , staircase=False, name=None)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_rate),loss='categorical_crossentropy',metrics=['acc'])

In [15]:
history=model.fit(x_train,y_train,batch_size=64,epochs=100,validation_data=(x_test,y_test),verbose=1)

Epoch 1/100
166/782 [=====>........................] - ETA: 19s - loss: 3.0059 - acc: 0.1413

KeyboardInterrupt: 